# Data Cleaning

This notebook loads the raw dataset, handles missing values, performs feature engineering (cleaning units from numerical columns), and saves the cleaned dataset for modeling.

In [1]:
import pandas as pd
import numpy as np
import kagglehub
import os

# Get dataset path
path = kagglehub.dataset_download("nehalbirla/vehicle-dataset-from-cardekho")
dataset_path = os.path.join(path, "car details v4.csv")
print("Loading:", dataset_path)

df = pd.read_csv(dataset_path)
df.head()

/home/miky/UNI/ML_Antigravity/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading: /home/miky/.cache/kagglehub/datasets/nehalbirla/vehicle-dataset-from-cardekho/versions/4/car details v4.csv


,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Engine,Max Power,Max Torque,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,Honda,Amaze 1.2 VX i-VTEC,505000,2017,87150,Petrol,Manual,Pune,Grey,First,Corporate,1198 cc,87 bhp @ 6000 rpm,109 Nm @ 4500 rpm,FWD,3990.0,1680.0,1505.0,5.0,35.0
1,Maruti Suzuki,Swift DZire VDI,450000,2014,75000,Diesel,Manual,Ludhiana,White,Second,Individual,1248 cc,74 bhp @ 4000 rpm,190 Nm @ 2000 rpm,FWD,3995.0,1695.0,1555.0,5.0,42.0
2,Hyundai,i10 Magna 1.2 Kappa2,220000,2011,67000,Petrol,Manual,Lucknow,Maroon,First,Individual,1197 cc,79 bhp @ 6000 rpm,112.7619 Nm @ 4000 rpm,FWD,3585.0,1595.0,1550.0,5.0,35.0
3,Toyota,Glanza G,799000,2019,37500,Petrol,Manual,Mangalore,Red,First,Individual,1197 cc,82 bhp @ 6000 rpm,113 Nm @ 4200 rpm,FWD,3995.0,1745.0,1510.0,5.0,37.0
4,Toyota,Innova 2.4 VX 7 STR [2016-2020],1950000,2018,69000,Diesel,Manual,Mumbai,Grey,First,Individual,2393 cc,148 bhp @ 3400 rpm,343 Nm @ 1400 rpm,RWD,4735.0,1830.0,1795.0,7.0,55.0


## Handling Missing Values

In [2]:
print("Missing values before cleaning:")
print(df.isnull().sum())

# Drop rows with missing critical values if they are few, or impute
# For this dataset, let's see. If 'Engine', 'Max Power' etc are missing, we might drop them or impute with mode/mean.
# Given the likely size, we'll drop rows with missing values for simplicity and data quality, 
# but in a production system we might use more sophisticated imputation.
df_cleaned = df.dropna()
print("\nShape after dropping nulls:", df_cleaned.shape)

Missing values before cleaning:
Make                    0
Model                   0
Price                   0
Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                 80
Max Power              80
Max Torque             80
Drivetrain            136
Length                 64
Width                  64
Height                 64
Seating Capacity       64
Fuel Tank Capacity    113
dtype: int64

Shape after dropping nulls: (1874, 20)


## Feature Cleaning & Engineering
We need to converting columns like 'Engine', 'Max Power' to numeric by removing units.

In [3]:
# Function to extract number from string
def clean_currency(x):
    if isinstance(x, str):
        return float(x.split()[0])
    return x

# Clean 'Engine' (remove ' cc')
df_cleaned['Engine'] = df_cleaned['Engine'].apply(lambda x: float(x.replace(' cc', '')) if isinstance(x, str) else x)

# Clean 'Max Power' (remove ' bhp' and handling other formats if any)
# Some entries might be 'bhp' or contain '@'
# Let's be robust: extract the first number
import re

def extract_bhp(x):
    if isinstance(x, str):
        # Search for a number (int or float) at the start
        match = re.search(r'^([\d\.]+)', x)
        if match:
            return float(match.group(1))
    return x

df_cleaned['Max Power'] = df_cleaned['Max Power'].apply(extract_bhp)

# Clean 'Max Torque' - this is complex (e.g. "190 Nm @ 2000 rpm")
# We will extract the Torque value (Nm)
def extract_torque(x):
    if isinstance(x, str):
        # Typical format: "190 Nm ..."
        # Extract first number
        match = re.search(r'^([\d\.]+)', x)
        if match:
            return float(match.group(1))
    return x

df_cleaned['Max Torque'] = df_cleaned['Max Torque'].apply(extract_torque)

# Show info after cleaning
df_cleaned.info()

<class 'pandas.DataFrame'>
Index: 1874 entries, 0 to 2057
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Make                1874 non-null   str    
 1   Model               1874 non-null   str    
 2   Price               1874 non-null   int64  
 3   Year                1874 non-null   int64  
 4   Kilometer           1874 non-null   int64  
 5   Fuel Type           1874 non-null   str    
 6   Transmission        1874 non-null   str    
 7   Location            1874 non-null   str    
 8   Color               1874 non-null   str    
 9   Owner               1874 non-null   str    
 10  Seller Type         1874 non-null   str    
 11  Engine              1874 non-null   float64
 12  Max Power           1874 non-null   float64
 13  Max Torque          1874 non-null   float64
 14  Drivetrain          1874 non-null   str    
 15  Length              1874 non-null   float64
 16  Width               18

## Calculate Car Age

In [4]:
import datetime
current_year = datetime.datetime.now().year

df_cleaned['Car_Age'] = current_year - df_cleaned['Year']
df_cleaned.head()

,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,...,Engine,Max Power,Max Torque,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity,Car_Age
0,Honda,Amaze 1.2 VX i-VTEC,505000,2017,87150,Petrol,Manual,Pune,Grey,First,...,1198.0,87.0,109.0000,FWD,3990.0,1680.0,1505.0,5.0,35.0,9
1,Maruti Suzuki,Swift DZire VDI,450000,2014,75000,Diesel,Manual,Ludhiana,White,Second,...,1248.0,74.0,190.0000,FWD,3995.0,1695.0,1555.0,5.0,42.0,12
2,Hyundai,i10 Magna 1.2 Kappa2,220000,2011,67000,Petrol,Manual,Lucknow,Maroon,First,...,1197.0,79.0,112.7619,FWD,3585.0,1595.0,1550.0,5.0,35.0,15
3,Toyota,Glanza G,799000,2019,37500,Petrol,Manual,Mangalore,Red,First,...,1197.0,82.0,113.0000,FWD,3995.0,1745.0,1510.0,5.0,37.0,7
4,Toyota,Innova 2.4 VX 7 STR [2016-2020],1950000,2018,69000,Diesel,Manual,Mumbai,Grey,First,...,2393.0,148.0,343.0000,RWD,4735.0,1830.0,1795.0,7.0,55.0,8


## Save Cleaned Data

In [5]:
output_path = "cleaned_car_data.csv"
df_cleaned.to_csv(output_path, index=False)
print(f"Cleaned data saved to {output_path}")

Cleaned data saved to cleaned_car_data.csv
